<a href="https://colab.research.google.com/github/Imesh7/ace-step/blob/main/ace_step_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision torchaudio soundfile numpy


# !pip install torch==2.8.0 torchvision torchaudio soundfile numpy

# !pip uninstall torchcodec
!pip install torchcodec==0.7

INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.9/887.9 MB 865.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 7.9 MB/s eta

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Install ffmpeg
!apt update
!apt install -y ffmpeg



Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://cli.github.com/packages stable/main amd64 Packages [356 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.8 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,888 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,682 kB]
Get:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:14 http://security.u

In [4]:
import torch.nn as nn
import torch
import torchaudio
import torchvision
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
from math import inf
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
from tqdm import tqdm
# import torchcodec
import random
import soundfile as sf

In [5]:


def get_mel_spectrogram(audio_path):
  waveform, sample_rate = sf.read(audio_path)
  waveform = torch.tensor(waveform, dtype=torch.float32).T
  mel_transform = torchaudio.transforms.MelSpectrogram()
  return mel_transform(waveform)


waveform, sample_rate = sf.read(
    '/content/drive/MyDrive/songs/Ahasa Se Oba Ananthai.wav'
)

# Convert to torch tensor
waveform = torch.tensor(waveform, dtype=torch.float32).T  # shape: (channels, samples) or (samples,) if mono
# print(waveform.shape)

# when you do the  'MelSpectrogram' pytorch internally do the STFT(short time fourier tranform) , so need to do it again
mel_transform = torchaudio.transforms.MelSpectrogram(
    # sample_rate=sample_rate,
    # n_fft=1024,
    # hop_length=256,
    # n_mels=80
    )
mel = mel_transform(waveform)

print(mel.shape)

/usr/local/lib/python3.12/dist-packages/torchaudio/functional/functional.py:585: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


torch.Size([2, 128, 62826])


In [6]:
# plot the mel-spectrogram graph

def render_melspectrogram(mel, title=None):
  plt.figure(figsize=(10, 4))

  # mel_to_plot = mel[0] # <-- select the index

  mel = mel.squeeze(0)
  mel_db = torch.log(mel[0])

  plt.figure(figsize=(10, 4))
  plt.imshow(
      mel_db.numpy(),
      origin="lower",
      aspect="auto"
  )
  plt.colorbar(label="Log Energy")
  plt.xlabel("Time Frames")
  plt.ylabel("Mel Frequency Bins")
  plt.title("Mel Spectrogram")
  plt.tight_layout()
  plt.show()


In [7]:
# Implement 'Deep Compression AutoEncoder'(DC-AE) to achieve highly compact mel-spectrogram latent representation

# https://arxiv.org/pdf/2410.10733

# pls refer
# 1. https://github.com/mit-han-lab/efficientvit
# 2. https://github.com/mit-han-lab/efficientvit/blob/master/efficientvit/models/efficientvit/dc_ae.py
# 3. https://github.com/mit-han-lab/efficientvit/blob/master/efficientvit/models/nn/ops.py

# You may asked why not the `SD-VAE` , I think because, when on high dimention I saw drops image quality (check out
# the above paper they figured out that issue)

class DeepCompressionAutoEncoder(nn.Module):
  def __init__(self,in_channels, *args, **kwargs) -> None:
    super().__init__(*args, **kwargs)
    self.encoder = Encoder(in_channels=in_channels, out_channels=32 * in_channels)
    self.decoder = Decoder(in_channels=32 * in_channels, out_channels=in_channels)

  def forward(self, x):
    x , skip_connections = self.encoder(x)
    x = self.decoder(x, skip_connections)
    return x


def conv_block(in_channels, out_channels):
  x = nn.Sequential(
    nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.BatchNorm2d(out_channels),
  )

  return x

class Encoder(nn.Module):
  def __init__(self, in_channels, out_channels, *args, **kwargs) -> None:
    super().__init__(*args, **kwargs)
    self.layers = nn.ModuleList()
    self.layers.append(conv_block(in_channels=in_channels, out_channels=out_channels))

  def forward(self, x):
    skip_connection = []
    for layer in self.layers:
      x = layer(x)
      skip_connection.append(x)
    return x, skip_connection



class Decoder(nn.Module):
  def __init__(self, in_channels, out_channels, *args, **kwargs) -> None:
    super().__init__(*args, **kwargs)
    self.layers = nn.ModuleList()
    self.layers.append(conv_block(in_channels=in_channels, out_channels=out_channels))

  def forward(self, x, skip_connections):
    skip_connection = skip_connections[::-1]
    for layer in self.layers:
      x = layer(x)
      x = torch.cat([x, skip_connection.pop(0)], dim=1)
    return x

In [8]:
# Implement DiT (Diffusion Transformer)

class DiffusionTransformer(nn.Module):
  def __init__(self, in_channels, out_channels, *args, **kwargs) -> None:
    super().__init__(*args, **kwargs)
    # Re-check the in_channels & out , because in multi atten we did divide the value by heads
    self.atten = MultiHeadAttention(in_channels=in_channels, out_channels=512, num_heads=8)
    self.norm = nn.LayerNorm(512)



  def sinusoidal_positional_encoding(self, max_length, dim):
    pos = np.arange(max_length)[:,np.newaxis]

    div = np.exp(np.arange(0, dim, 2) * -np.log(10000) * dim)
    print(div)

    emb = np.zeros([max_length, dim])
    emb[:, 0::2] = np.sin(pos / div)
    emb[:, 1::2] = np.cos(pos / div)
    return emb


  def forward(self, x):
    # first flatten the latent
    x = x.flatten(start_dim=2)

    pos = self.sinusoidal_positional_encoding(x.shape[1], x.shape[2])
    x_pos = x + pos
    x_atten = self.atten(x_pos)
    return x_atten


class MultiHeadAttention(nn.Module):
  def __init__(self, in_channels, out_channels, num_heads, *args, **kwargs) -> None:
    super().__init__(*args, **kwargs)
    self.in_channels = in_channels
    self.out_channels = out_channels
    self.num_heads = num_heads
    self.d_model = in_channels
    self.head_dim = in_channels // num_heads
    # multiplied by 3 , beacuse we have 'q' , 'k' & 'v' seperations
    self.qkv = nn.Linear(in_features=in_channels, out_features= 3 * out_channels)
    self.proj = nn.Linear(in_features=out_channels, out_features=out_channels)


  def forward(self, x):
    batch_size, samples, emb_dim = x.shape
    # In here we have a Tensor Like [5, 12, 3*512]
    # this is like 5 batch, 12 samples, 3 * 512 (dimentios)
    qkv = self.qkv(x)

    # Lets reshape it
    qkv = qkv.view(batch_size, samples, 3, self.num_heads, self.head_dim)
    qkv = qkv.permute(2 , 0 , 3 , 1 , 4)

    query, key, value = qkv

    mask = torch.tril(torch.ones(samples, self.head_dim))
    x = F.scaled_dot_product_attention(query, key, value, atten_mask=mask)

    proj = self.proj(x)
    return proj


class FeedForward(nn.Module):
  def __init__(self, in_channels, out_channels):
    self.layer1 = nn.Conv1d(in_channels=in_channels, out_channels=out_channels)
    self.layer2 = nn.Conv1d(in_channels=in_channels, out_channels=3 * out_channels)
    self.act = nn.SiLU()
    self.layer3 = nn.Conv1d(in_channels=3 * in_channels, out_channels=out_channels)


  def forward(self, x):
    x = self.layer1(x)
    x = self.layer2(x)
    input = x

    x = self.act(x)
    x = x + input
    return self.layer3(x)


# Implement cross attention for the TAGS, LY
class CrossAttention(nn.Module):
  def __init__(self, *args, **kwargs):
    super().__init__(*args, **kwargs)

In [9]:
import os
import torchaudio
from torch.utils.data import Dataset

class DriveAudioDataset(Dataset):
    def __init__(self, root_dir, n_mels=80, sample_rate=22050, target_length_sec=5.0):
        self.root_dir = root_dir
        self.files = [
            f for index , f in zip(range(2), os.listdir(root_dir))
            if f.endswith(".wav")
        ]
        self.target_samples = int(target_length_sec * sample_rate)

        self.mel_transform = torchaudio.transforms.MelSpectrogram(
            sample_rate=sample_rate,
            n_mels=n_mels,
            n_fft=1024,
            hop_length=256,
            power=2.0
        )

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        path = os.path.join(self.root_dir, self.files[idx])
        waveform, sample_rate = sf.read(path)

        if waveform.ndim == 2 and waveform.shape[1] == 2:
          waveform = waveform.mean(axis=1)

        waveform = torch.tensor(waveform, dtype=torch.float32).unsqueeze(0)  # [1, samples]

        # Randome crop
        if waveform.shape[-1] > self.target_samples:
            # Random start point
            start = random.randint(0, waveform.shape[-1] - self.target_samples)
            waveform = waveform[:, start:start + self.target_samples]
        else:
            # Pad if too short
            pad_len = self.target_samples - waveform.shape[-1]
            waveform = torch.nn.functional.pad(waveform, (0, pad_len))

        mel_spectrogram = self.mel_transform(waveform)
        return mel_spectrogram

In [10]:
# DC-AE

#sp - Encoder is only need in Training , decoder is needed training & inference as well.

# Train
def train():
  epochs = 5
  dataset = DriveAudioDataset('/content/drive/MyDrive/songs/')
  dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
  model = DeepCompressionAutoEncoder(in_channels=1)

  for epoch in range(epochs):
    for batch_idx, data in enumerate(dataloader):
      print(f"Batch {batch_idx}: Data shape {data.shape}")
      # render_melspectrogram(data)
      output = model(data)

      optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

      optimizer.zero_grad()
      loss = F.mse_loss(output, data)
      loss.backward()
      optimizer.step()
      print(f"epoch {epoch}, Loss: {loss.item()}")



# Inference


train()

Batch 0: Data shape torch.Size([1, 1, 80, 431])


/tmp/ipython-input-2100397604.py:21: UserWarning: Using a target size (torch.Size([1, 1, 80, 431])) that is different to the input size (torch.Size([1, 33, 80, 431])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(output, data)


epoch 0, Loss: 112745.21875
Batch 1: Data shape torch.Size([1, 1, 80, 431])
epoch 0, Loss: 282271.0
Batch 0: Data shape torch.Size([1, 1, 80, 431])
epoch 1, Loss: 268967.03125
Batch 1: Data shape torch.Size([1, 1, 80, 431])
epoch 1, Loss: 373488.25
Batch 0: Data shape torch.Size([1, 1, 80, 431])
epoch 2, Loss: 408829.40625
Batch 1: Data shape torch.Size([1, 1, 80, 431])
epoch 2, Loss: 100185.2890625
Batch 0: Data shape torch.Size([1, 1, 80, 431])
epoch 3, Loss: 10003.80078125
Batch 1: Data shape torch.Size([1, 1, 80, 431])
epoch 3, Loss: 112445.640625
Batch 0: Data shape torch.Size([1, 1, 80, 431])
epoch 4, Loss: 197203.078125
Batch 1: Data shape torch.Size([1, 1, 80, 431])
epoch 4, Loss: 229397.203125


In [11]:
# DiT

#Train

# Get the saved 'VAE' model
# Add noise to the mel spectrogram & train

class CosineSchedule():
  def __init__(self, Timestamp):
    s = 0.008
    # Compute f(t) for all timesteps
    steps = torch.arange(Timestamp + 1, dtype=torch.float32)
    ft = torch.cos(((steps / Timestamp + s) / 1 + s) * (np.pi / 2)) ** 2
    self.alpha_bar = ft / ft[0]

    self.alphas = self.alphas_bar[1:] / self.alphas_bar[:-1]
    self.betas = 1 - self.alphas
    self.sqrt_alphas_bar = torch.sqrt(self.alphas_bar)
    self.sqrt_one_minus_alphas_bar = torch.sqrt(1 - self.alphas_bar)

  def noise_schedule(self, t, x0, noise=None):
    sqrt_alpha = self.sqrt_alphas_bar[t].view(-1, 1, 1, 1)
    sqrt_one_minus = self.sqrt_one_minus_alphas_bar[t].view(-1, 1, 1, 1)

    if noise is None:
      noise = torch.randn_like(x0)
    return sqrt_alpha * x0 + sqrt_one_minus * noise


def train_model():
  epochs = 5
  timesteps = 100
  dataset = DriveAudioDataset('/content/drive/MyDrive/songs/')
  dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
  model = DeepCompressionAutoEncoder(in_channels=1)
  cosine_schedule = CosineSchedule(Timestamp=timesteps)
  dit = DiffusionTransformer(in_channels=80)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

  for epoch in range(epochs):
    optimizer.zero_grad()
    for batch_idx, data in enumerate(dataloader):
      print(f"Batch {batch_idx}: Data shape {data.shape}")
      latent = model(data)
      timestep = torch.randn(timesteps)

      noise_input = cosine_schedule.noise_schedule(timestep, latent)
      latent_pred = dit(noise_input, timestep)
      loss = F.l1_loss(latent_pred, latent)
      loss.backward()
      optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
      optimizer.step()
      print(f"epoch {epoch}, Loss: {loss.item()}")



# Inference

# Remove noise
